In [3]:
import requests
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
import numpy as np
import datetime as dt
from pathlib import Path
import os
import csv
%matplotlib inline

In [4]:
load_dotenv()

True

In [36]:
csv_path = Path("../Yolanda_Baker_Notebook/ticker_revenue.csv")
revenue_data = pd.read_csv(csv_path, index_col="Date", parse_dates=True, infer_datetime_format=True)
revenue_data.tail(9)

,Ticker,Revenue
Date,,
2018-01-01,YUM,5688.0
2019-01-01,YUM,5597.0
2020-01-01,YUM,5652.0
2018-01-01,GRUB,1007.2
2019-01-01,GRUB,1312.1
2020-01-01,GRUB,1819.9
2018-01-01,MCD,21257.9
2019-01-01,MCD,21364.4
2020-01-01,MCD,19207.8


In [37]:
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")
alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version="v2")

In [38]:
timeframe = "1D"
# Set start and end datetimes between now and 3 years ago.
start_date = pd.Timestamp("2018-01-01", tz="America/New_York").isoformat()
end_date = pd.Timestamp("2018-12-31", tz="America/New_York").isoformat()

# Set the ticker information
tickers = ["YUM", "GRUB", "MCD"]

# Get 3 year's worth of historical price data for Microsoft and Coca-Cola
returns2018_data = alpaca.get_barset(
    tickers,
    timeframe,
    start=start_date,
    end=end_date,
    limit=1000,
).df

# Display sample data
returns2018_data.head()

GRUB                                     MCD  \
                            open   high      low  close   volume    open   
2018-01-02 00:00:00-05:00  71.54  72.01  70.2600  71.27  1255728  173.73   
2018-01-03 00:00:00-05:00  71.45  71.78  70.9100  71.31   714615  173.23   
2018-01-04 00:00:00-05:00  71.93  72.60  70.9900  71.87   843995  173.24   
2018-01-05 00:00:00-05:00  72.43  73.02  71.3796  72.96  1126306  174.00   
2018-01-08 00:00:00-05:00  71.50  72.10  70.8200  71.33   959352  173.74   

                                                                YUM         \
                               high     low   close   volume   open   high   
2018-01-02 00:00:00-05:00  174.4799  172.66  173.22  2344135  82.15  82.46   
2018-01-03 00:00:00-05:00  173.6400  172.00  172.49  2706678  81.23  81.82   
2018-01-04 00:00:00-05:00  174.1300  172.73  173.75  1860820  81.75  82.59   
2018-01-05 00:00:00-05:00  175.0000  173.40  174.04  2031490  82.59  83.01   
2018-01-08 00:00:00-05:00  174.2100  172.93  173.92  1401313  82.72  83.06   

                                                  
                             low  close   volume  
2018-01-02 00:00:00-05:00  81.48  81.61  1089380  
2018-01-03 00:00:00-05:00  81.17  81.53  1923085  
2018-01-04 00:00:00-05:00  81.53  82.35  1585861  
2018-01-05 00:00:00-05:00  82.36  82.84  1542610  
2018-01-08 00:00:00-05:00  82.12  82.96  1309538

In [41]:
project_df = pd.DataFrame()
project_df["YUM"] = returns2018_data["YUM"]["close"]
project_df["GRUB"] = returns2018_data["GRUB"]["close"]
project_df["MCD"] = returns2018_data["MCD"]["close"]
project_df.index = project_df.index.date
annual2018_returns = project_df.pct_change().dropna()
annual2018_returns.head(10)

,YUM,GRUB,MCD
2018-01-03,-0.000980,0.000561,-0.004214
2018-01-04,0.010058,0.007853,0.007305
2018-01-05,0.005950,0.015166,0.001669
2018-01-08,0.001449,-0.022341,-0.000689
2018-01-09,-0.002290,-0.030282,-0.002242
2018-01-10,-0.002658,-0.038311,-0.000058
2018-01-11,-0.000848,0.025556,-0.000864
2018-01-12,0.014064,0.035327,0.001038
2018-01-16,-0.008608,-0.010760,0.000691
2018-01-17,0.001930,0.013310,0.007428


In [42]:
annual2018_returns.describe()

,YUM,GRUB,MCD
count,250.000000,250.000000,250.000000
mean,0.000537,0.000868,0.000162
std,0.012369,0.035962,0.012963
min,-0.075378,-0.119226,-0.047848
25%,-0.004750,-0.014725,-0.005455
50%,0.001243,0.000828,0.000002
75%,0.007164,0.016437,0.006691
max,0.045802,0.275082,0.062995


In [43]:
annual2018_returns.std()

YUM     0.012369
GRUB    0.035962
MCD     0.012963
dtype: float64

In [28]:
project_df = project_df.pct_change(251).dropna()

In [44]:
annual2018_returns.std()

YUM     0.012369
GRUB    0.035962
MCD     0.012963
dtype: float64

In [45]:
standard_dev = pd.DataFrame(annual2018_returns.std()).rename(columns = {0: "2019 STD"})
standard_dev

,2019 STD
YUM,0.012369
GRUB,0.035962
MCD,0.012963
